# Project Group - 21

Members: Naomi Cornelissen, Tom Bruinink, Sam Buisman, Bart van Heijningen, Klaas Molendijk

Student numbers: 5105765, 6075673, 5957915, 6088015, 

# Research Objective


Background information:
Trains are generally seen as an environmental friendly mode of transport. The European Union recently announced a plan to improve the railway network in Europe in order to reduce CO2 emissions . The trend of investing in railway transport begs the question whether those investments are effective. Because there is often a great amount of money involved in those investments, this research takes a deeper dive into the relationship between investments in the rail network for passengers and the usage of that train network. The goal is to create a visual overview of that relationship. The infrastructure investments are measured relative to the gross domestic product (GDP) of a country. 
Research question:
What is the influence of a country’s investments in railway passenger transport relative to the GDP on train travel?
Datasets:
1.	Amount of passenger-kilometres per country (2000-2020)
https://data.oecd.org/transport/passenger-transport.htm#indicator-chart 
2.	GDP (per capita) per country (2000-2020) https://databank.worldbank.org/indicator/NY.GDP.PCAP.CD/1ff4a498/Popular-Indicators# https://ourworldindata.org/grapher/gdp-per-capita-maddison?tab=table
3.	Investments in railway passenger transport per country (2000-2020)
https://data.oecd.org/transport/infrastructure-investment.htm#indicator-chart


# Contribution Statement

*Be specific. Some of the tasks can be coding (expect everyone to do this), background research, conceptualisation, visualisation, data analysis, data modelling*

**Author 1**:

**Author 2**:

**Author 3**:

# Data Used

# Data Pipeline